1\. Write a function that converts number representation (bin<->dec<->hex)

In [30]:

def rappr_to_10(x,base):
    num=0
    power=len(x)-1
    for c in x:
        if c=="A":
            c=10
        elif c=="B":
            c=11
        elif c=="C":
            c=12
        elif c=="D":
            c=13
        elif c=="E":
            c=14
        elif c=="F":
            c=15

        if int(c)>=base:
            return "il numero non e' valido"
        num+=base**power*int(c)
        power -= 1
    return str(num)
    
def convert_from_10(x,base):
    num=int(x)
    l=""
    while num>=base:
        resto=num%base
        num=int((num-resto)/base)
        
        l=str(lettere(resto))+l #la stringa aggiunge il numero davanti non in fondo
    resto=num%base
        
    l=str(lettere(resto))+l
    num=(num-resto)/base
    return l
        
    
def lettere(x):
    if x<10:
        y=x
    elif x==10:
        y="A"
    elif x==11:
        y="B"
    elif x==12:
        y="C"
    elif x==13:
        y="D"
    elif x==14:
        y="E"
    elif x==15:
        y="F"
    return y

print(convert_from_10("163",15))
print(rappr_to_10("AD",15))
print(convert_from_10(rappr_to_10("AD",15),2))
print(rappr_to_10("11111111111111111111111",2))

AD
163
10100011
8388607


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [1]:
num=3333333333
mask_sgn=2147483648
mask_exp=2139095040
mask_mant=8388607

sgn=num & mask_sgn
exp= num & mask_exp
mant= num & mask_mant


sgn=sgn>>31   #sposto i bit di 31 posizioni a destra
#print(sgn)
exp>>=23 #sposto i bit di 23 posizioni a destra
#print(exp)

#non c'è bisogno di shiftare mant

exp-=127 #devo togliere il bias all'esponente

mantnew=(mant+2**23)/(2**23) #devo ricordarmi che la matissa è definita con un 1,... davanti che corrisponde a un 2^23

n=(-1)**sgn*(((mantnew)*2**(exp)))

    
print(n)

-22352.666015625


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [11]:
uf=1
of=1

def underflow(x):
    i=0
    while x!=0:
        x/=2
        i+=1
    return i


print("underflow limit is ", uf/(2**(underflow(uf)-1)))


def overflow(x):
    i=0
    y=float(x)
    while y<y*2:
        y*=2
        i+=1
    return i

f=float(2**(overflow(of)-1))
    

print("overflow limit is ", f)


underflow limit is  5e-324
overflow limit is  8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [19]:
n=1
eps=1

while n!=n+eps:
    eps/=2
    
print(eps*2) #devo mopltiplicare per 2 perché serve il penultimo epsilon perché l'ultimo è quello che diventa irrisorio

2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [30]:

def sol(a,b,c):
    x1=(-b+(b**2-4*a*c)**(0.5))/(2*a)

    x2=(-b-(b**2-4*a*c)**(0.5))/(2*a)

    print("x1=",x1)
    print("x2=",x2)
    
sol(0.001,1000,0.001)

def sol1(a,b,c):
    x1=(4*a*c)/(2*a*(-b-(b**2-4*a*c)**(0.5)))

    x2=(4*a*c)/(2*a*(-b+(b**2-4*a*c)**(0.5)))

    print("x1=",x1)
    print("x2=",x2)
    
sol1(0.001,1000,0.001)

def solperfect(a,b,c):
    if b>0:
        x2=(-b-(b**2-4*a*c)**(0.5))/(2*a)
        x1=(4*a*c)/(2*a*(-b-(b**2-4*a*c)**(0.5)))
    else:
        x2=(-b-(b**2-4*a*c)**(0.5))/(2*a)
        x1=(-b+(b**2-4*a*c)**(0.5))/(2*a)

    print("x1=",x1)
    print("x2=",x2)
    
solperfect(0.001,1000,0.001)

x1= -9.999894245993346e-07
x2= -999999.999999
x1= -1.0000000000009999e-06
x2= -1000010.5755125057
x1= -1.0000000000009999e-06
x2= -999999.999999


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [4]:
#nel primo caso delta è troppo grande
def f(x):
    return x*(x-1)
for i in range(2,15,2):
    delta=10**(-i)
    derive=(f(1+delta)-f(1))/delta
    print("delta=",10**(-i),"risultato=",derive, "accuratezza=",derive-1)


delta= 0.01 risultato= 1.010000000000001 accuratezza= 0.010000000000000897
delta= 0.0001 risultato= 1.0000999999998899 accuratezza= 9.999999988985486e-05
delta= 1e-06 risultato= 1.0000009999177333 accuratezza= 9.99917733279787e-07
delta= 1e-08 risultato= 1.0000000039225287 accuratezza= 3.922528746258536e-09
delta= 1e-10 risultato= 1.000000082840371 accuratezza= 8.284037100736441e-08
delta= 1e-12 risultato= 1.0000889005833413 accuratezza= 8.890058334132256e-05
delta= 1e-14 risultato= 0.9992007221626509 accuratezza= -0.0007992778373491216


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt{(1-x^2)} {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [11]:
def f(x):
    return (1-x**2)**(0.5)




def integral(N):
    I=0
    h=2/N
    for i in range(N):
        I+=h*f(i*h-1)
    return I
        

print('the value of the integral is: ', integral(1000000), 'accuracy:',abs(integral(1000000)-1.57079632679))


the value of the integral is:  1.5707963251317274 accuracy: 1.6582726303226991e-09


In [14]:
print('the value of the integral is for 1s running: ', integral(394500), 'accuracy:',abs(integral(394500)-1.57079632679))
%timeit integral(394500)


the value of the integral is for 1s running:  1.5707963200829944 accuracy: 6.7070056175566606e-09
1.04 s ± 41.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
print('the value of the integral is for 1 min running: ', integral(23670000), 'accuracy:',abs(integral(23670000)-1.57079632679))


the value of the integral is for 1 min running:  1.5707963267803133 accuracy: 9.686695889854491e-12
